In [ ]:
!pip install pydub

In [ ]:
import os
from pydub import AudioSegment

In [ ]:
trimmed_mp3 = r"/content/drive/MyDrive/Confidence Checker Project/trimmed mp3"

# WAV Converter

In [ ]:
def batch_convert_to_wav(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".mp3"):
            mp3_path = os.path.join(directory, filename)
            try:
                audio = AudioSegment.from_mp3(mp3_path)
                wav_path = mp3_path.replace(".mp3", ".wav")
                audio.export(wav_path, format="wav")
                print(f"Converted: {filename} to WAV format")
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                print(f"Skipping {filename}...")

batch_convert_to_wav(trimmed_mp3)


# WAV Files Mover

In [ ]:
import shutil

def move_wav_files(source_directory, target_directory):
    os.makedirs(target_directory, exist_ok=True)

    for filename in os.listdir(source_directory):
        if filename.endswith(".wav"):  # Check for .wav files
            source_path = os.path.join(source_directory, filename)
            target_path = os.path.join(target_directory, filename)
            try:
                shutil.move(source_path, target_path)  # Move the file
                print(f"Moved: {filename} to {target_directory}")
            except Exception as e:
                print(f"Error moving {filename}: {e}")

# Specify source and target directories
source_dir = trimmed_mp3
target_dir = r"/content/drive/MyDrive/Confidence Checker Project/wav"

move_wav_files(source_dir, target_dir)


# Audio Trimmer

In [ ]:
directory_mp3 = r"/content/drive/MyDrive/Confidence Checker Project/mp3"

In [ ]:
import os
from pydub import AudioSegment

def trim_audio(input_file, output_file, duration_limit=180):
    audio = AudioSegment.from_file(input_file)
    duration = len(audio) / 1000  # duration in seconds

    if duration > duration_limit:
        trimmed_audio = audio[duration_limit * 1000:]  # Trim first 180 seconds
        trimmed_audio.export(output_file, format="mp3")
        print(f"Trimmed {input_file} to {output_file}")
    else:
        print(f"{input_file} is shorter than {duration_limit} seconds. Skipping.")

def process_directory(input_directory, output_directory, duration_limit=180):
    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    for filename in os.listdir(input_directory):
        if filename.endswith(".mp3"):
            input_file = os.path.join(input_directory, filename)
            output_file = os.path.join(output_directory, filename)  # Save in new folder
            trim_audio(input_file, output_file, duration_limit)


output_directory = r"/content/drive/MyDrive/Confidence Checker Project/trimmed mp3"

# Run the function
process_directory(directory_mp3, output_directory)


# Feature Extractor

In [ ]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 19.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803320 sha256=9d5015c962a8b635464544c9f9439e8e1bc74452325b9ecdbb0d0de8de132b88
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [ ]:
pip install webrtcvad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp310-cp310-linux_x86_64.whl size=73458 sha256=3334a1566046febe03ea161980ce15e670212ee258623493f9acd5e0db097733
  Stored in directory: /root/.cache/pip/wheels/2a/2b/84/ac7bacfe8c68a87c1ee3dd3c66818a54c71599abf308e8eb35
Successfully built webrtcvad


# Essential Libraries

In [ ]:
import os
import re
from concurrent.futures import ThreadPoolExecutor
from collections import Counter
import whisper
import pandas as pd
import librosa
import numpy as np
import warnings
import webrtcvad

In [ ]:
directory = "/content/drive/MyDrive/Confidence Checker Project/wav"

# Data Loader

In [ ]:
def load_audio_data(directory, target_sr=22050, mono=True):

    audio_file_dict = {}
    for filename in os.listdir(directory):
        if filename.lower().endswith(".wav"):
            file_path = os.path.join(directory, filename)
            try:
                # Load the audio file with librosa, specifying resampling rate and mono conversion
                audio_data, sample_rate = librosa.load(file_path, sr=target_sr, mono=mono)

                # Ensure audio data is in the correct format, i.e., floating point
                if not np.issubdtype(audio_data.dtype, np.floating):
                    audio_data = audio_data.astype(np.float32)

                audio_file_dict[filename] = {
                    "audio_data": audio_data,
                    "sample_rate": sample_rate
                }
                print(f"Loaded and processed {filename}: {len(audio_data)} samples at {sample_rate} Hz, mono={mono}")

            except Exception as e:
                print(f"Error loading or processing {filename}: {e}")
        else:
            print(f"Skipped {filename} as it is not a WAV file.")
    return audio_file_dict


# Check if Noise Reduction Required

In [ ]:
def detect_noise_level(audio, sample_rate, noise_threshold=0.01):
    rms_energy = np.sqrt(np.mean(audio**2))
    return rms_energy < noise_threshold

# Noise Reduction

In [ ]:
def reduce_noise(audio, sample_rate, noise_clip_duration=0.5):
    try:
        noise_clip_samples = int(noise_clip_duration * sample_rate)
        noise_clip = audio[:noise_clip_samples]

        audio_stft = librosa.stft(audio, n_fft=2048, hop_length=512, win_length=2048)
        noise_stft = librosa.stft(noise_clip, n_fft=2048, hop_length=512, win_length=2048)

        noise_mag = np.mean(np.abs(noise_stft), axis=1, keepdims=True)

        audio_mag = np.abs(audio_stft)
        audio_phase = np.angle(audio_stft)
        denoised_mag = np.maximum(audio_mag - noise_mag, 0)

        denoised_stft = denoised_mag * np.exp(1j * audio_phase)
        denoised_audio = librosa.istft(denoised_stft, hop_length=512, win_length=2048)

        return denoised_audio

    except Exception as e:
        print(f"Error in noise reduction: {e}")
        return audio


# Transcribe Audio Simpler Way

In [ ]:
import whisper
import numpy as np

def transcribe_audio(audio, sample_rate, segment_duration=30):
    """
    Transcribe audio robustly by splitting into manageable segments if needed.
    """
    try:
        model = whisper.load_model("large")
    except Exception as e:
        print(f"Failed to load Whisper model: {e}")
        return ""

    segment_samples = segment_duration * sample_rate
    segments = [audio[i * segment_samples:(i + 1) * segment_samples] for i in range(len(audio) // segment_samples + (len(audio) % segment_samples != 0))]

    full_transcript = []
    for segment in segments:
        try:
            result = model.transcribe(np.array(segment), language="en", beam_size=5, fp16=True)
            transcript = result.get('text', '').strip()
            full_transcript.append(transcript)
        except Exception as e:
            print(f"Error transcribing segment: {e}")

    return " ".join(full_transcript)


#Repeated Words Counter

In [ ]:
def detect_repeated_words(transcript, min_repeats=2):
    try:
        normalized_text = re.sub(r'[^\w\s]', '', transcript.lower())
        words = normalized_text.split()

        repeated_words = []
        for i in range(len(words) - 1):
            if words[i] == words[i + 1]:
                repeated_words.append(words[i])

        repeated_counts = Counter(repeated_words)

        # Filter by minimum repeats
        filtered_repeats = {word: count for word, count in repeated_counts.items() if count >= min_repeats}

        return {
            "total_repeated_words": len(repeated_words),
            "repeated_words_counts": filtered_repeats
        }

    except Exception as e:
        print(f"Error detecting repeated words: {e}")
        return {
            "total_repeated_words": 0,
            "repeated_words_counts": {}
        }


# Pause Extractor

In [ ]:
from pydub import AudioSegment

def preprocess_audio(input_path, output_path):
    audio = AudioSegment.from_file(input_path)
    mono_audio = audio.set_channels(1).set_frame_rate(16000)  # Convert to mono and resample to 16 kHz
    mono_audio.export(output_path, format="wav")
    return output_path

In [ ]:
import wave
import numpy as np
import webrtcvad

def extract_pauses_from_wav(wav_file_path, vad_mode=3, pause_threshold=0.5, frame_duration=10):
    try:
        # Open WAV file
        with wave.open(wav_file_path, 'rb') as wf:
            num_channels = wf.getnchannels()
            sample_width = wf.getsampwidth()
            sample_rate = wf.getframerate()

            # Validate audio format
            if num_channels != 1:
                raise ValueError("Audio must be mono.")
            if sample_width != 2:
                raise ValueError("Audio must be 16-bit PCM.")
            if sample_rate != 16000:
                raise ValueError("Audio must have a sample rate of 16 kHz.")

            # Read frames and convert to int16 NumPy array
            audio = wf.readframes(wf.getnframes())
            audio = np.frombuffer(audio, dtype=np.int16)

        # Initialize WebRTC VAD
        vad = webrtcvad.Vad(vad_mode)

        # Compute frame size in samples
        frame_size = int(sample_rate * frame_duration / 1000)

        # Create frames
        frames = [
            audio[i:i + frame_size]
            for i in range(0, len(audio) - frame_size + 1, frame_size)
        ]
        if len(audio) % frame_size != 0:
            last_frame = audio[len(audio) - len(audio) % frame_size:]
            frames.append(np.pad(last_frame, (0, frame_size - len(last_frame))))

        # Process frames with VAD
        pause_durations = []
        start_silence = None
        for i, frame in enumerate(frames):
            is_speech = vad.is_speech(frame.tobytes(), sample_rate)
            if not is_speech:
                if start_silence is None:
                    start_silence = i
            else:
                if start_silence is not None:
                    pause_duration = (i - start_silence) * frame_duration / 1000
                    if pause_duration >= pause_threshold:
                        pause_durations.append(pause_duration)
                    start_silence = None

        # Calculate pause features
        total_pauses = len(pause_durations)
        avg_pause_duration = np.mean(pause_durations) if pause_durations else 0.0

        return {
            "total_pauses": total_pauses,
            "pause_durations": pause_durations,
            "average_pause_duration": avg_pause_duration,
        }

    except Exception as e:
        print(f"Error extracting pauses: {e}")
        return {
            "total_pauses": 0,
            "pause_durations": [],
            "average_pause_duration": 0.0,
        }


# Average Pitch Extractor

In [ ]:
def extract_pitch(audio_data, sample_rate):
    chunk_size = sample_rate * 10  # e.g., 10 seconds
    num_chunks = len(audio_data) // chunk_size + (len(audio_data) % chunk_size != 0)
    avg_pitches = []

    for i in range(num_chunks):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, len(audio_data))
        chunk = audio_data[start:end]
        pitches, _ = librosa.piptrack(y=chunk, sr=sample_rate)
        pitch_values = pitches[pitches > 0]
        pitch_values = pitch_values[(pitch_values > 50) & (pitch_values < 300)]
        avg_pitch = np.mean(pitch_values) if pitch_values.size > 0 else 0
        avg_pitches.append(avg_pitch)

    overall_avg_pitch = np.mean(avg_pitches) if avg_pitches else 0
    return overall_avg_pitch


# Pitch Variability

In [ ]:
import numpy as np
import librosa

def calculate_pitch_variability(audio_data, sr, min_pitch=50, max_pitch=500):
    pitches, magnitudes = librosa.piptrack(y=audio_data, sr=sr)
    refined_pitches = []

    for i in range(pitches.shape[1]):
        pitch_frame = pitches[:, i]
        mag_frame = magnitudes[:, i]
        if mag_frame.max() > 0:
            max_pitch_idx = mag_frame.argmax()
            pitch = pitch_frame[max_pitch_idx]
            if min_pitch <= pitch <= max_pitch:
                refined_pitches.append(pitch)

    refined_pitches = np.array(refined_pitches)
    pitch_variability = np.std(refined_pitches) if refined_pitches.size > 0 else 0
    return pitch_variability


# Speech Rate

In [ ]:
def calculate_speech_rate(audio_data, sample_rate, min_duration=0.2):
    duration = librosa.get_duration(y=audio_data, sr=sample_rate)
    if duration < min_duration:
        return 0

    try:
        model = whisper.load_model("base")
        result = model.transcribe(audio_data)
        words = result['text'].split()
        words_count = len(words)
        speech_rate = (words_count / duration) * 60
        return speech_rate
    except Exception as e:
        print(f"Error calculating speech rate: {e}")
        return 0


# Filler Words Extracter + Counter

In [ ]:

def count_filler_words(transcript, filler_words):
    """Counts the total occurrences of filler words in a transcript."""
    if not transcript or not filler_words:
        return 0

    word_list = re.findall(r'\b\w+\b', transcript.lower())
    word_counts = Counter(word_list)
    total_fillers = sum(word_counts.get(word, 0) for word in filler_words)

    print(f"Total filler words found: {total_fillers}")
    return total_fillers



# DF Creation

In [ ]:
fillers = [
    "um", "uh", "like", "you know", "so", "actually", "basically",
    "literally", "seriously", "honestly", "I mean", "well", "right",
    "kind of", "sort of", "I guess", "probably", "maybe", "for sure",
    "at the end of the day", "the thing is", "you see", "I think",
    "obviously", "as I said", "essentially",
    "anyway", "to be honest", "just", "okay", "anyhow", "oh"
]

In [ ]:
def extract_features_from_audio(directory, fillers):
    audio_data = load_audio_data(directory)  # Assumes this loads {filename: {audio_data, sample_rate}}
    features_dict = {}

    for filename, data in audio_data.items():
        try:
            if 'audio_data' not in data or 'sample_rate' not in data:
                print(f"Missing required keys in {filename}. Skipping.")
                continue

            audio = data['audio_data']
            sample_rate = data['sample_rate']

            # Apply Noise Reduction
            if detect_noise_level(audio, sample_rate):
                audio = reduce_noise(audio, sample_rate)

            features = {"file_name": filename}

            # Extract features
            transcript = transcribe_audio(audio, sample_rate)

            features["filler_words"] = count_filler_words(transcript, fillers)
            features["avg_pitch"] = extract_pitch(audio, sample_rate)
            features["pitch_variability"] = calculate_pitch_variability(audio, sample_rate)
            features["speech_rate"] = calculate_speech_rate(audio, sample_rate)

            # Detect repeated words
            repeated_words_features = detect_repeated_words(transcript)
            features["total_repeated_words"] = repeated_words_features["total_repeated_words"]

            # Generate label
            file_label = os.path.splitext(filename)[0]
            features["label"] = re.sub(r'\d+|\(.*?\)', '', file_label).strip()

            # # Pause extraction
            # preprocessed_path = preprocess_audio(os.path.join(directory, filename), os.path.join(directory, "preprocessed", filename))
            # results = extract_pauses_from_wav(preprocessed_path)
            # features["total_pauses"] = results["total_pauses"]
            # features["pause_durations"] = results["pause_durations"]
            # features["average_pause_duration"] = results["average_pause_duration"]

            # Collect features
            for key, value in features.items():
                if key not in features_dict:
                    features_dict[key] = []
                features_dict[key].append(value)

        except Exception as e:
            print(f"Error processing {filename}: {e}")

    return features_dict

In [ ]:
def save_features_to_csv(features_dict, directory):
    try:
        features_df = pd.DataFrame(features_dict)
        output_file = os.path.join(directory, "audio_features_3.csv")
        features_df.to_csv(output_file, index=False)
        print(f"Feature extraction complete. Data saved to {output_file}")
    except Exception as e:
        print(f"Failed to save CSV file: {e}")

In [ ]:
features_dict = extract_features_from_audio(directory, fillers)
save_features_to_csv(features_dict, directory)

Loaded and processed Unconfident (50).wav: 71641376 samples at 22050 Hz, mono=True
Loaded and processed Confident (51).wav: 63178785 samples at 22050 Hz, mono=True
Loaded and processed Confident (50).wav: 157936146 samples at 22050 Hz, mono=True
Loaded and processed Confident (49).wav: 8909854 samples at 22050 Hz, mono=True
Loaded and processed Confident (48).wav: 95448607 samples at 22050 Hz, mono=True
Loaded and processed Confident (47).wav: 58103646 samples at 22050 Hz, mono=True
Loaded and processed Confident (46).wav: 56823776 samples at 22050 Hz, mono=True
Loaded and processed Confident (45).wav: 41474529 samples at 22050 Hz, mono=True
Loaded and processed Confident (44).wav: 62332947 samples at 22050 Hz, mono=True
Loaded and processed Confident (43).wav: 32808019 samples at 22050 Hz, mono=True
Loaded and processed Confident (42).wav: 53860829 samples at 22050 Hz, mono=True
Loaded and processed Confident (40).wav: 8859051 samples at 22050 Hz, mono=True
Loaded and processed Confid

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1941
  warnings.warn(


Error extracting pauses: Error while processing frame
Total filler words found: 171
Error extracting pauses: Error while processing frame
Total filler words found: 88
Error extracting pauses: Error while processing frame
Total filler words found: 113
Error extracting pauses: Error while processing frame
Total filler words found: 490
Error extracting pauses: Error while processing frame
Total filler words found: 10
Error extracting pauses: Error while processing frame
Total filler words found: 4
Error extracting pauses: Error while processing frame
Total filler words found: 11
Error extracting pauses: Error while processing frame
Total filler words found: 9
Error extracting pauses: Error while processing frame
Total filler words found: 56
Error extracting pauses: Error while processing frame
Total filler words found: 250
Error extracting pauses: Error while processing frame
Total filler words found: 55
Error extracting pauses: Error while processing frame
Total filler words found: 2
Err

# Updating CSV with Pause Analysis

In [ ]:
def update_csv_with_pauses(csv_file_path, audio_directory, output_csv_path):
    try:
        df = pd.read_csv(csv_file_path)

        total_pauses = []
        pause_durations = []
        average_pause_duration = []

        for _, row in df.iterrows():
            filename = row["file_name"]
            audio_path = os.path.join(audio_directory, filename)

            if not os.path.exists(audio_path):
                print(f"File not found: {audio_path}. Skipping.")
                total_pauses.append(None)
                pause_durations.append(None)
                average_pause_duration.append(None)
                continue

            preprocessed_path = preprocess_audio(audio_path, os.path.join(audio_directory, "preprocessed", filename))
            results = extract_pauses_from_wav(preprocessed_path)

            total_pauses.append(results["total_pauses"])
            average_pause_duration.append(results["average_pause_duration"])

        df["total_pauses"] = total_pauses
        df["average_pause_duration"] = average_pause_duration

        label_index = df.columns.get_loc("label")
        cols_to_move = ["total_pauses", "average_pause_duration"]
        df = pd.concat([df.iloc[:, :label_index], df[cols_to_move], df.iloc[:, label_index:]], axis=1)

        df.to_csv(output_csv_path, index=False)
        print(f"CSV updated successfully. Saved to {output_csv_path}")

    except Exception as e:
        print(f"Error updating CSV: {e}")


In [ ]:
update_csv_with_pauses(
    "/content/drive/MyDrive/Confidence Checker Project/wav/audio_features_2.csv",
    "/content/drive/MyDrive/Confidence Checker Project/wav",
    "/content/drive/MyDrive/Confidence Checker Project/wav/updated_audio_features.csv"
)

CSV updated successfully. Saved to /content/drive/MyDrive/Confidence Checker Project/wav/updated_audio_features.csv


# Model Preparation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Confidence Checker Project/wav/updated_audio_features.csv")

In [ ]:
df.head()

,file_name,filler_words,avg_pitch,pitch_variability,speech_rate,total_pauses,average_pause_duration,total_repeated_words,total_pauses.1,average_pause_duration.1,label
0,Unconfident (50).wav,564,229.736350,76.556190,139.241044,722,1.363241,78,722,1.363241,Unconfident
1,Confident (51).wav,114,224.475540,87.833790,172.738792,535,0.863290,11,535,0.863290,Confident
2,Confident (50).wav,1108,218.803757,99.742485,157.701695,1552,1.850638,233,1552,1.850638,Confident
3,Confident (49).wav,10,216.499908,71.874880,117.601927,82,1.542927,3,82,1.542927,Confident
4,Confident (48).wav,1708,215.075851,111.396940,201.037947,795,0.814440,74,795,0.814440,Confident


In [ ]:
df.drop(['total_pauses.1','average_pause_duration.1'], axis=1, inplace=True)

In [ ]:
df.head()

,file_name,filler_words,avg_pitch,pitch_variability,speech_rate,total_pauses,average_pause_duration,total_repeated_words,label
0,Unconfident (50).wav,564,229.736350,76.556190,139.241044,722,1.363241,78,Unconfident
1,Confident (51).wav,114,224.475540,87.833790,172.738792,535,0.863290,11,Confident
2,Confident (50).wav,1108,218.803757,99.742485,157.701695,1552,1.850638,233,Confident
3,Confident (49).wav,10,216.499908,71.874880,117.601927,82,1.542927,3,Confident
4,Confident (48).wav,1708,215.075851,111.396940,201.037947,795,0.814440,74,Confident


In [ ]:
X = df[['filler_words','avg_pitch','pitch_variability','speech_rate','total_pauses','average_pause_duration','total_repeated_words']]
y = df['label']

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=88, stratify=y_encoded)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=49)

In [ ]:
scores = cross_val_score(model, X, y, cv=10)

# Print results
print("Accuracy scores for each fold:", scores)
print("Mean cross-validation accuracy:", scores.mean())
print("Standard deviation of accuracy across folds:", scores.std())

Accuracy scores for each fold: [0.8        0.8        0.8        0.8        0.8        0.77777778
 0.88888889 0.88888889 0.88888889 0.66666667]
Mean cross-validation accuracy: 0.8111111111111111
Standard deviation of accuracy across folds: 0.06382847385042252


In [ ]:
model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=49)

In [ ]:
y_pred = model.predict(X_test_scaled)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))

Accuracy: 0.8421052631578947

Classification Report:
               precision    recall  f1-score   support

           0       0.84      1.00      0.91        16
           1       0.00      0.00      0.00         3

    accuracy                           0.84        19
   macro avg       0.42      0.50      0.46        19
weighted avg       0.71      0.84      0.77        19



In [ ]:
print("\nLabel Mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))
# reversed_mapping = {label: 1 - label_encoder.transform([label])[0] for label in label_encoder.classes_}
# print("Label Mapping:", reversed_mapping)


Label Mapping: {'Confident': 0, 'Unconfident': 1}


In [ ]:
from joblib import dump, load

In [ ]:
dump(model, "model_joblib_V2.joblib")

['model_joblib_V2.joblib']

In [ ]:
import pickle

with open("model_pkl_V2.pkl", "wb") as file:
    pickle.dump(model, file)